In [1]:
import numpy as np
import pandas as pd
import feather
import torch
import torchvision
import datetime
import os
from six.moves import range

In [2]:
data_root ='D:\\Programming\\Projects\\notMNIST'
image_size=28

def feather_3d(file,image_size = 28):
    temp = feather.read_dataframe(file)
    width = np.shape(temp.values)[1]
    if width == 1:
        final = temp.values
        final = final.reshape(-1)
    else:
        final = np.reshape(temp.values, (-1, image_size, image_size))
    return final

def read_data(dir, filename):
    file = os.path.join(dir, filename + '.feather')
    output = feather_3d(file)
    return output

test_dataset = read_data(data_root, 'test_dataset_final')
test_labels = read_data(data_root, 'test_labels_final')
train_dataset = read_data(data_root, 'train_dataset_final')
train_labels = read_data(data_root, 'train_labels_final')
valid_dataset = read_data(data_root, 'valid_dataset_final')
valid_labels = read_data(data_root, 'valid_labels_final')

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197309, 28, 28) (197309,)
Validation set (17460, 28, 28) (17460,)
Test set (10202, 28, 28) (10202,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, num_channels, image_size, image_size)).astype(np.float32)
  labels = labels.reshape(-1,1).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197309, 1, 28, 28) (197309, 1)
Validation set (17460, 1, 28, 28) (17460, 1)
Test set (10202, 1, 28, 28) (10202, 1)


In [4]:
batch_size = 32

def numpy_to_torch(dataset, label, batch_size, shuffle=False, num_workers=2):
    tensor_dataset = torch.stack([torch.tensor(i) for i in dataset])
    tensor_label = torch.stack([torch.tensor(i) for i in label])
    tensor_label = tensor_label.long()
    my_dataset = torch.utils.data.TensorDataset(tensor_dataset,tensor_label)
    return torch.utils.data.DataLoader(my_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

trainloader=numpy_to_torch(train_dataset, train_labels, batch_size,shuffle=True, num_workers=3)
testloader=numpy_to_torch(test_dataset, test_labels, batch_size, num_workers=3)
validloader=numpy_to_torch(valid_dataset, valid_labels, batch_size, num_workers=3)

In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(device)

net.to(device)

cuda:0


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,3], gamma=0.1)

In [7]:
starttime = datetime.datetime.now()

for epoch in range(4):  # loop over the dataset multiple times
    scheduler.step()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels.view(-1))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            correct = 0
            total = 0
            with torch.no_grad():
                for data in validloader:
                    images, labels = data
                    labels = labels.view(-1)
                    outputs = net(images.to(device))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels.to(device)).sum().item()
            print('[%d, %5d] loss: %.3f, and valid_acc is %.1f %%' %
                  (epoch + 1, i + 1, running_loss / 2000, 100 * correct / total))
            running_loss = 0.0

print('Finished Training')

endtime = datetime.datetime.now()
print(endtime-starttime)

[1,  2000] loss: 0.643, and valid_acc is 84.4 %
[1,  4000] loss: 0.489, and valid_acc is 84.3 %
[1,  6000] loss: 0.507, and valid_acc is 79.7 %
[2,  2000] loss: 0.532, and valid_acc is 83.1 %
[2,  4000] loss: 0.521, and valid_acc is 83.9 %
[2,  6000] loss: 0.514, and valid_acc is 82.8 %
[3,  2000] loss: 0.404, and valid_acc is 86.8 %
[3,  4000] loss: 0.371, and valid_acc is 87.2 %
[3,  6000] loss: 0.360, and valid_acc is 87.5 %
[4,  2000] loss: 0.343, and valid_acc is 87.7 %
[4,  4000] loss: 0.338, and valid_acc is 87.7 %
[4,  6000] loss: 0.333, and valid_acc is 87.8 %
Finished Training
0:02:53.504381


In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = labels.view(-1)
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 93 %


In [9]:
classes = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J')

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = labels.view(-1)
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels.squeeze().to(device))
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     A : 93 %
Accuracy of     B : 94 %
Accuracy of     C : 95 %
Accuracy of     D : 94 %
Accuracy of     E : 93 %
Accuracy of     F : 94 %
Accuracy of     G : 94 %
Accuracy of     H : 93 %
Accuracy of     I : 88 %
Accuracy of     J : 93 %
